In [ ]:
import pandas as pd
import time
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part, SafetySetting
import vertexai.preview.generative_models as generative_models

from dotenv import load_dotenv
import os
load_dotenv()



In [ ]:
input_file = ""
output_file = ''

In [ ]:
vertex_project = os.environ['VERTEX_PROJECT_ID']

In [ ]:
def generate(prompt):
  vertexai.init(project=vertex_project, location="us-central1")
  model = GenerativeModel("gemini-1.5-pro-002")
  responses = model.generate_content(
    prompt,
    generation_config={
        "max_output_tokens": 1000,
        "temperature": 0.4,
        "top_p": 1
    },
    safety_settings = [
        SafetySetting(
            category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
            threshold=SafetySetting.HarmBlockThreshold.OFF
        ),
        SafetySetting(
            category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
            threshold=SafetySetting.HarmBlockThreshold.OFF
        ),
        SafetySetting(
            category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
            threshold=SafetySetting.HarmBlockThreshold.OFF
        ),
        SafetySetting(
            category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
            threshold=SafetySetting.HarmBlockThreshold.OFF
        ),
    ],
    stream=False,
  )
  
  return responses

In [ ]:
df = pd.read_csv(input_file)


print(len(df))
print()
df.head()

In [ ]:
def construct_prompt(fato_relevante, analise):
    resp = f'''### Instruções:
    Você receberá dois documentos: "Documento 1" e "Documento 2". Sua tarefa é analisar o conteúdo de ambos e determinar a natureza da relação entre eles. 
    
    ### Opções de Resposta:
    Referência Explícita: O "Documento 2" cita explicitamente os mesmos eventos específicos tratados no "Documento 1", seja através de citações diretas ou paráfrases claras.
    Sem Referência: O "Documento 2" não faz referência alguma aos eventos específicos tratados no "Documento 1".

    ### Formato de Saída:
    resposta: [Referência Explícita/Sem Referência]
    trechos que confirmem a resposta: [Trechos relevantes de ambos os documentos que suportam sua resposta]
    
    ### Documento 1
    {fato_relevante}
    
    ### Documento 2
    {analise}'''
    
    return resp

In [ ]:
import re
import unicodedata

def remover_acentos(texto):
    texto_normalizado = unicodedata.normalize('NFD', texto)
    return ''.join([c for c in texto_normalizado if not unicodedata.combining(c)])

def verificar_referencia(texto):
    texto = remover_acentos(texto)
    
    match = re.search(r"resposta:\s*(.*?)\s*(trechos que|$)", texto, re.IGNORECASE)
    if not match:
        return "Nenhum padrão de resposta encontrado."
    
    resposta = match.group(1).strip().lower()
    
    if "referencia explicita" in resposta:
        return "referencia explicita"
    elif "referencia implicita" in resposta:
        return "referencia implicita"
    elif "referencia complementar" in resposta:
        return "referencia complementar"
    elif "referencia especulativa" in resposta:
        return "referencia especulativa"
    elif "sem referencia" in resposta:
        return "sem referencia"
    else:
        return -1

In [ ]:
answers = ["none"] * len(df)
y_pred = [-1] * len(df)

In [ ]:
i = 0

while i < len(df):
  try:
    prompt = construct_prompt(df["AN_conteudo"].iloc[i], df["FR_conteudo"].iloc[i])

    resp = generate(prompt)

    resp_texto = resp.text.replace("\n", '')
    answers[i] = resp_texto.lower()
    y_pred[i] = verificar_referencia(resp_texto)
    
    i += 1

    if i % 50 == 0:
        print("Progresso: ", i)

    # Pausa de 2 segundos (sugerido é 1 solicitação a cada 5s)
    time.sleep(2)

  except Exception as e:
    erro = f"Erro encontrado: {e}. Tentando novamente..."
    print(erro)
    
    answers[i] = erro

    i += 1
    
    time.sleep(5)  # Pausa antes de tentar novamente

In [ ]:
import numpy as np

errors = [i for i, v in enumerate(y_pred) if isinstance(v, (int, float)) and v == -1]

print(errors)

In [ ]:
checkpoint = y_pred

In [ ]:
for i in errors:
  try:
    prompt = construct_prompt(df["pdf_text"].iloc[i], df["dates"].iloc[i], df["content"].iloc[i], df["fr_date2"].iloc[i])

    resp = generate(prompt)

    resp_texto = resp.text.replace("\n", '')
    answers[i] = resp_texto.lower()
    y_pred[i] = verificar_referencia(resp_texto)
    
    i += 1

    if i % 50 == 0:
        print("Progresso: ", i)

    # Pausa de 2 segundos (sugerido é 1 solicitação a cada 5s)
    time.sleep(3)

  except Exception as e:
    erro = f"Erro encontrado: {e}. Tentando novamente..."
    print(erro)
    
    answers[i] = erro

    i += 1
    
    time.sleep(5)  # Pausa antes de tentar novamente

In [ ]:
errors = [i for i, v in enumerate(y_pred) if isinstance(v, (int, float)) and v == -1]

print(errors)

In [ ]:
print("Contagem de 'referencia explicita':", y_pred.count("referencia explicita"))
print("Contagem de 'referencia implicita':", y_pred.count("referencia implicita"))
print("Contagem de 'referencia complementar':", y_pred.count("referencia complementar"))
print("Contagem de 'referencia especulativa':", y_pred.count("referencia especulativa"))
print("Contagem de 'sem referencia':", y_pred.count("sem referencia"))
print("Contagem de respostas não classificadas (-1):", y_pred.count(-1))

In [ ]:
df["pred"] = y_pred
df["responses_raw"] = answers

In [ ]:
df.to_csv(output_file, index=False)